# Covid long et offre de travail

Projet réalisé en 2025 par Lucile Aubain, Jean Lavallée et Paul Hobeika dans le cadre du cours de *Python pour la data science* enseigné en deuxième année de l'ENSAE.

# Introduction
<a id="introduction"></a>

La pandémie de Covid-19 a contaminé près de 800 millions de personnes et entraîné environ sept millions de décès dans le monde entre 2020 et 2025, dont 150 000 en France. Au-delà de ces conséquences sanitaires directes, la pandémie a engendré des effets durables sur les trajectoires individuelles et sur le fonctionnement du marché du travail. Ce projet s'intéresse à une dimension encore peu étudiée en France sous l'angle de l'économie de la santé et du travail : la hausse persistante des arrêts de travail pour maladie depuis 2020 et ses implications pour l'offre de travail de long terme.

La littérature médicale documente désormais largement l'existence de séquelles durables chez une fraction non négligeable des personnes infectées par le ovid-19, y compris après des formes initialement bénignes. Ces affections de long terme, regroupées sous l'appellation de « Covid long », recouvrent des symptômes variés – fatigue chronique, troubles cognitifs ou neurologiques, difficultés respiratoires – susceptibles de limiter durablement la capacité à exercer une activité professionnelle ([Groff et al., 2021](https://pubmed.ncbi.nlm.nih.gov/34643720/), [Ford et al., 2025](https://onlinelibrary.wiley.com/doi/abs/10.1002/ajim.70014?casa_token=NAKQSBiFLa8AAAAA%3AIUjd4EXrl4jU1-7FkTjwP6W7jjW2aly-mTo0LDxL27Oh4bVbCj11uAcFBwdYeCaNNGGWgzkhD-0v5TfP), [MacEwan et al., 2025](https://link.springer.com/article/10.1007/s11606-024-09062-5)).

Ces constats ont nourri une littérature économique récente qui analyse le rôle du Covid long dans la contraction de l'offre de travail observée après la pandémie. À partir de données américaines, Goda et al. ([2023](https://www.sciencedirect.com/science/article/pii/S0047272723000713)) montrent que les travailleurs absents au moins une semaine en 2020 pour raisons liées au Covid-19 ont une probabilité inférieure de 7 % d'être toujours actifs un an plus tard, contribuant à un déficit estimé à environ 500 000 emplois aux États-Unis. D'autres travaux mettent également en évidence des sorties durables de la population active associées au Covid long ([Evans et al., 2021](https://www.thelancet.com/journals/lanres/article/PIIS2213-2600\(21\)00383-0/fulltext?uuid=uuid%3A4cfb08b5-99b4-49e0-ab9b-14f97739bbb6), [Ziauddeen et al., 2022](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0264331)). À partir de données néerlandaises, Bussink et al. ([2025](https://pubmed.ncbi.nlm.nih.gov/41180946/)) estiment qu'une infection au Covid-19 réduit d'environ 1 point de pourcentage la probabilité d'emploi des nouveaux entrants sur le marché du travail dans les mois suivant la diplomation, un effet plus limité que dans les études portant sur l'ensemble de la population adulte, possiblement en raison d'une moindre prévalence du Covid long chez les jeunes.

Cette littérature s'inscrit plus largement dans les débats relatifs aux « travailleurs manquants » (*missing workers*) observés dans de nombreuses économies avancées après la pandémie. Au Royaume-Uni, Reuschke et Houston ([2022](https://www.tandfonline.com/doi/full/10.1080/13504851.2022.2098239)) estiment qu'environ 80 000 personnes ont quitté leur emploi en raison du Covid long, contribuant aux pénuries de main-d'œuvre. Ces résultats suggèrent que les effets macroéconomiques de la pandémie pourraient être sensiblement sous-estimés si l'on ne tient pas compte de ses conséquences sanitaires de long terme.

Plusieurs institutions ont ainsi avancé des ordres de grandeur importants quant à l'impact du Covid long sur l'offre de travail. Un rapport de la *Brookings Institution* ([2022](https://www.brookings.edu/articles/is-long-covid-worsening-the-labor-shortage/)) estime que jusqu'à quatre millions d'Américains pourraient être éloignés de l'emploi en raison du Covid long, soit environ 15 % des postes vacants en 2022. Dans l'Union européenne, la Commission européenne ([2024](https://economy-finance.ec.europa.eu/document/download/36713cbb-6cbf-4ddb-8a15-55a4f456e2cb_en?filename=eb077_en.pdf)) évalue la perte d'offre de travail associée au Covid long entre 0,2 et 0,3 % en 2021 et entre 0,3 et 0,5 % en 2022, correspondant à 600 000 à 1 million de personnes. Des estimations comparables sont proposées pour l'ensemble des pays de l'OCDE ([Abraham et Rendell, 2023](https://www.brookings.edu/wp-content/uploads/2023/03/BPEA_Spring2023_Abraham-Rendell_unembargoed.pdf), [Espinosa Gonzalez et Suzuki, 2024](https://www.oecd.org/content/dam/oecd/en/publications/reports/2024/06/the-impacts-of-long-covid-across-oecd-countries_f662b21c/8bd08383-en.pdf)). À ce titre, le Covid long pourrait porter le coût économique total de la pandémie à près de 1 000 milliards de dollars pour l'OCDE ([Espinosa Gonzalez et Suzuki, 2024](https://www.oecd.org/content/dam/oecd/en/publications/reports/2024/06/the-impacts-of-long-covid-across-oecd-countries_f662b21c/8bd08383-en.pdf)).

Dans ce contexte, ce travail étudie l'effet de l'exposition au Covid-19 en 2020 sur l'offre de travail de long terme des salariés en France, à partir des données de l'Enquête emploi en continu (EEC). En l'absence d'une mesure directe de l'exposition individuelle au virus dans l'enquête, nous proposons une méthode indirecte permettant de classer les secteurs et catégories socioprofessionnelles selon leur degré d'exposition en 2020. Nous faisons l'hypothèse que le taux d'arrêts de travail pour maladie observé en 2020 constitue un *proxy* pertinent de l'exposition au Covid-19 pour une catégorie donnée. Le contexte de crise sanitaire, marqué par l'arrêt de la production dans certains secteurs et par les mesures de confinement, a en effet conduit à une surexposition de certaines professions, tandis que d'autres en étaient relativement protégées ([Dubost, Pollak et Rey, 2020](https://drees.solidarites-sante.gouv.fr/sites/default/files/2020-10/DD62.pdf)).

Nos résultats montrent que les salariés des secteurs les plus exposés en 2020 réduisent durablement leur offre de travail, bien au-delà du pic de la pandémie. Une analyse d'événement indique que les travailleurs des secteurs fortement exposés ont une probabilité inférieure d'environ X points de pourcentage d'être en activité un an plus tard, relativement à des travailleurs comparables des secteurs peu exposés. À plus long terme, en 2024, DÉVELOPPER. Ce travail contribue à la littérature existante en apportant des éléments nouveaux pour le cas français, encore peu documenté, et en mettant en évidence un canal sanitaire susceptible d'expliquer une partie des difficultés de recrutement observées dans certains secteurs à partir de 2021.

Ce travail est organisé de la façon suivante. La section [Installation](#installation) présente les librairies et fonctions utilisées. La section [Données](#donnees) décrit les données et leur préparation, avant la construction d'un indicateur d'exposition sectorielle dans la section [Exposition au Covid-19 en 2020](#exposition). La section [Covid long et réduction de l'offre de travail](#modelisation) analyse les effets de long terme de l'exposition au Covid-19 sur l'offre de travail. La section [Conclusion](#conclusion) conclut.

# Sommaire
<a id="sommaire"></a>

- [Introduction](#introduction)
- [Installation](#installation)
- [Données](#donnees)
- [Statistiques descriptives](#descriptives)
- [Covid long et réduction de l'offre de travail](#modelisation)
- [Conclusion](#conclusion)

# Installation
<a id="installation"></a>

In [1]:
%%capture
%pip install -r scripts/requirements.txt

# Modules
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import requests
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from lets_plot import *
# from palmerpenguins import load_penguins #  supprimer ? Je pense pas qu'on l'utilise
from urllib import request
from io import BytesIO
# from sklearn.cluster import KMeans # on n'utilise pas non a priori ?
import linearmodels as lm
from linearmodels.panel import PanelOLS
import seaborn as sns
import importlib
import statsmodels.formula.api as smf
import statsmodels.api as sm
from patsy.contrasts import Treatment


# Fonctions
from scripts import import_eec
from scripts import import_eec_all
from scripts import hospitalisations
from scripts import exposition

# Données
<a id="donnees"></a>

Dans le cadre de ce travail, nous analysons les données relatives aux absences issues des fichiers détails de l'Enquête emploi en continu (EEC). Un travailleur est enregistré comme absent s'il déclare à la fois être en activité et n'avoir effectué aucune heure de travail durant la semaine de référence de l'EEC. De plus, la « raison principale » de leur absence est demandée à tous les travailleurs absents (variable `RABS`). Parmi les dix motifs pouvant être invoqués par les employés figure « Congé maladie (y compris arrêt après un accident au travail) » (`RABS = 2`).

Cette variable est présente dans les fichiers détails de l'Enquête Emploi, accessibles en *open data* directement depuis le site de l'Insee (mais pas depuis la librairie python `pynsee`). Les fichiers étant publiés chaque année, l'import des données implique d'importer séparément chacune des années d'intérêt, puis d'assembler les différentes tables. Nous avons choisi d'utiliser les tables depuis 2010 jusqu'en 2024.

L'une des difficultés est que les variables peuvent varier selon les années : certaines variables peuvent changer de nom, de nomenclature voire disparaître d'une année sur l'autre. Par exemple, la variable d'âge est en cinq modalités et s'appelle `AGE5` en 2018 et 2019 puis elle passe à six modalités avec un découpage différent à partir de 2020 et s'appelle alors `AGE6`.

La plupart des variables qui nous intéressent restent cependant inchangées entre 2010 et 2024, en particulier : 

- la variable indiquant le statut d'activité des enquêtés (`ACTEU`), permettant de distinguer les actifs en emploi (`ACTEU = '1'`) du reste de la population.
- la variable permettant d'identifier les actifs déclarant n'avoir pas travaillé durant la semaine de référence pour cause de maladie (`RABS = '2'`).
- l'année de l'enquête (`ANNEE`).

D'autre variables dont nous avons besoin changent de nom (mais pas de nomenclature) selon les années. Lors de l'importation des données, nous renommons donc les variables suivantes afin de pouvoir les conserver dans la table résultant de la concaténation de toutes les fichiers détail de 2010 à 2024 : 

- la variable de catégorie socioprofessionnelle à 2 chiffres : présente sous des noms différents, nous la renommons `CSE` pour l'ensemble des années
- la variable de secteur d'activité à 38 modalités : `NAFG38UN`

La fonction `load_eec_all()` importe les différentes bases de données annuelles puis les assemble en une base de données multiannuelle en conservant toutes les variables présentes avec le même nom toutes les années. Pour éviter d'avoir à faire systématiquement cette phase d'importation qui est longue (en particulier pour les fichier `DBF` utilisés de 2010 à 2017), nous avons enregistré la base obtenue dans un fichier `PARQUET` que l'on peut importer directement. Pour importer les fichier directement depuis le site de l'Insee : décommenter la ligne `eec_all = import_eec_all.load_eec_all()` et commenter la dernière ligne `eec_all = pd.read_parquet("https://minio.lab.sspcloud.fr/phobeika/open_eec/eec_all.parquet")`.

In [2]:
# Import des données #
######################
 
# Méthode 1 (lente, prend environ 5 minutes) : téléchargement direct depuis le 
# site de l'Insee ou via les url backup en cas d'erreur
#
# eec_all = import_eec_all.load_eec_all()
#
# Sauvegarde locale pour usage ultérieur
#
# object_cols = eec_all.select_dtypes(include='object').columns
# eec_all[object_cols] = eec_all[object_cols].astype(str)
# eec_all.to_parquet('data/eec_all.parquet', index=False)



# Méthode 2 (rapide) : chargement depuis un fichier obtenu via la méthode 1
# sauvegardé sur l'espace de stockage du SSP Cloud

eec_all = pd.read_parquet("https://minio.lab.sspcloud.fr/phobeika/open_eec/eec_all.parquet")

Selon les années, les données importées n'ont pas exactement le même format. Pour éviter les erreurs liées à des petites différences entre les années (par exemple des modalités `1` et `1.0` pour des variables catégorielles qui ont été à un moment enregistrées comme valeurs numériques), on utilise la fonction `recodages()` qui permet d'éviter ces problèmes en plus que de distinguer les variables selon leur type (catégoriel ou numérique).

In [3]:
# Recodages
eec_all = import_eec.recodages(
    eec_all, 
    vars_cat=['RABS', 'ACTEU', 'SEXE', 'CSE', 'PUB3FP'], 
    vars_num=['ANNEE']
    )

Enfin, on filtre l'ensemble des données de manière à ne travailler que sur la population active et ajouter des variables ` labels` pour rendre les modalités lisibles dans les tableaux et graphiques.

In [4]:
# Filter pour ne conserver que les actifs en emploi
eec_actifs = eec_all[eec_all['ACTEU'] == "1"].copy()

# Ajout de variables 'labels' pour rendre les modalités lisibles
eec_actifs = import_eec.add_labels(eec_actifs)

# Exposition au Covid-19 en 2020
<a id="exposition"></a>

Cette section vise à construire une variable d'exposition au Covid-19 en 2020 à partir des données sur les arrêts de travail. Cette utilisation ne va pas de soi, car nous voulons ensuite mesurer l'effet de ce score sur les arrêts de travail pour maladie dans les années qui suivent. Il s'agit donc d'utiliser les arrêts de travail à la fois comme variable explicative et comme variable d'intérêt. Pour justifier la pertinence de cette méthode, nous faisons une double hypothèse D'abord, nous faisons l'hypothèse que la pandémie de Covid-19 a été la cause principale d'arrêts pour maladie en 2020. Ensuite, nous supposons que même si le virus a continué à circuler les années suivantes, son effet sur la santé des actifs a été à la fois moindre (notamment en raison de la vaccination) et davantage "concurrencé" par d'autres maladies, si bien que la dynamique épidémique du Covid-19 est moins prédictive des arrêts pour maladie après 2020.

Cette section vise à justifier ces deux hypothèses et à construire la variable d'exposition que nous utiliserons dans la suite. Nous construisons cet indicateur à partir des variables de secteur d'activité et de catégorie socioprofessionnelle. Pour cela, nous commençons par étudier la variabilité des arrêts de travail selon ces catégories. Nous analysons ensuite la corrélation entre les arrêts de travail et les données d'hospitalisation liées au Covid-19 afin de justifier nos hypothèse de travail. Enfin, nous construisons un indicateur d'exposition en cinq catégories et vérifions sa cohérence avec les résultats de la littérature épidémiologique en étudiant la composition des différentes catégories ainsi construites.

### Les absences pour congé maladie en 2020 comme proxy de la contamination par le Covid-19

Nous faisons l'hypothèse que, dans le contexte pandémique de l'année 2020, les absence au travail pour maladie sont fortement corrélées à la circulation du Covid-19. Pour vérifier cette hypothèse, nous étudions ici la corrélation entre l'intensité pandémique mesurée à partir du nombre d'hospitalisations pour Covid-19 et les arrêts de travail pour maladie. Après avoir récupéré les données des hospitalisations liées au Covid-19 en 2020 via une API de Santé publique France, nous comparons l'évolution du nombre d'hospitalisations et d'arrêts maladies.

Tout d'abord, pour récupérer les données concernant le nombre d'hospitalisations liées au Covid-19, nous avons utilisé l'API Huwise de Santé publique France, présente sur le portail [open data Odissé](https://odisse.santepubliquefrance.fr/explore/dataset/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep/api/?sort=date). Après une lecture extensive de la [documentation de l'API](https://help.opendatasoft.com/apis/ods-explore-v2/) afin de trouver une solution pour contourner la faible limite de résultats par requête, nous avons exporté l'ensemble de la base "Covid-19 - Synthèse des indicateurs de suivi de la pandémie", *via* les commandes indiquées dans la documentation (regroupée dans la fonction `hospitalisations()`).

In [5]:
# Importation des données de Santé Publique France
hosp = hospitalisations.import_hosp(
    url_main="https://odisse.santepubliquefrance.fr/api/explore/v2.1/catalog/datasets/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep/exports/json",
    url_backup="https://minio.lab.sspcloud.fr/phobeika/open_eec/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep.json"
)

Tentative de téléchargement depuis: https://odisse.santepubliquefrance.fr/api/explore/v2.1/catalog/datasets/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep/exports/json
Importation réussie.


Pour pouvoir grouper et sommer les observations par date, nous avons sélectionné les données de 2020 en s'assurant qu'il existe toujours le même nombre de départements observés pour chaque date (101). Nous avons ensuite représenté graphiquement le nombre d'hospitalisations en fonction du temps, avec des étiquettes mensuelles pour plus de lisibilité


In [ ]:
hospitalisations.plot_hosp_year(hosp, year = 2020)

On retrouve bien les deux "vagues" caractéristiques du Covid en France en 2020. Il reste toutefois à savoir si les arrêts maladie suivent la même tendance que ces données d'hospitalisation. Comme l'échelle temporelle la plus fine contenue dans les fichiers détail des enquêtes emploi est le trimestre (plutôt que la journée pour les données d'hospitalisation), nous utilisons les quatre trimestres de 2020 pour comparer l'évolution des deux variables.

In [ ]:
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year=2020)

La corrélation entre le nombre d'arrêts de travail pour congé maladie et le nombre d'hospitalisations liées au Covid est de 0.51, ce qui est convenable pour un *proxy*, d'autant plus qu'une partie des personnes qui se seraient peut-être mis en arrêt de travail étaient déja arrêtées (chômage partiel...), réduisant les variations du nombre d'arrêts de travail.

Il reste enfin à vérifier que la proportion d'arrêts de travail pour maladie n'est plus corrélée pour les période suivantes. Nous faisons donc les mêmes graphiques pour les années suivantes.

In [ ]:
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year = 2021)
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year = 2022)
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year = 2023)

Ces graphiques montrent que la corrélation *positive* observée en 2020 entre les deux variables ne se reproduit pas dans les années qui suivent. Cela semble ainsi valider notre seconde hypothèse. On pourrait néanmoins discuter de la pertinence d'utiliser le nombre d'hospitalisations comme mesure de l'intensité de l'épidémie de Covid-19, car les campagnes de vaccination ont largement réduit les formes graves du virus, sans nécessairement réduire de la même proportion sa transmission et ses effets sur les arrêts de travail. On se limitera néanmoins à cette variable dans le cadre de ce projet, car l'échelle temporelle du trimestre limite quoi qu'il arrive la portée de cette analyse de corrélation. Pour aller plus loin, nous pourrions avoir recours aux données de l'enquête emploi diffusées par [Progedo](https://www.progedo.fr/la-serie-des-enquetes-emploi/) (qui permettent d'identifier la semaine de référence), que l'on pourrait mettre en parallèle avec la mesure de la concentration en virus dans les eaux usées (données [Sum'eaux](https://www.data.gouv.fr/datasets/surveillance-du-sars-cov-2-dans-les-eaux-usees-sumeau)).

### Des inégalités structurelles dans la probabilité d'être en arrêt maladie

Si la variable d'arrêt de travail constitue un *proxy* de l'exposition au Covid-19 en 2020, elle ne peut s'utiliser qu'à un niveau agrégé. En effet, les données ne permettent pas de suivre des individus dans le temps, si bien qu'il n'est pas possible d'identifier un effet d'être en arrêt de travail en 2020 sur les arrêts de travail ultérieurs *à une échelle individuelle*. Il est toutefois possible d'agréger les données selon des catégories plus ou moins exposées au virus en 2020. On vérifie ici que les variables de catégorie socioprofessionnelle et de secteur d'activité permettant bien de distinguer différentes catégories d'actifs plus ou moins exposés.

In [ ]:
# Calcul de la proportion d'actifs en arrêt de travail selon les années en fonction de la PCS
df_pcs_annee = exposition.exposition_annee(eec_actifs, var="CSER_label", year_col="ANNEE")

# Représentation graphique de l'évolution de la proportion d'arrêts maladie par catégorie socioprofessionnelle
exposition.plot_evolution_proportion(
    df=df_pcs_annee,
    year_col="ANNEE",
    group_col="CSER_label",
    value_col="proportion_rabs2",
    title="Évolution de la proportion d'absences au travail pour congé maladie selon la PCS",
    ylabel="Pourcentage d'actifs absents au travail pour congé maladie"
    )

Les catégories socioprofessionnelles agrégées permettent bien de distinguer les actifs selon le pourcentage d'arrêt de travail parmi chaque catégorie. De manière générale, les groupes d'ouvriers et d'employés sont ceux pour lesquels la proportion d'actifs en arrêt est la plus forte (entre 4 et 6 %), tandis que les catégories d'indépendants et les cadres sont en arrêt moins fréquemment (entre 1 et 2 % sur toute la période). Les professions intermédiaires se situent entre ces deux groupes (entre 2 et 3,5 %). 

Les différences structurelles entre catégories ne sont toutefois pas ce qui nous intéresse ici, puisqu'on cherche à estimer l'effet différentiel de l'exposition au Covid-19 en 2020. Cet effet ne correspond plutôt à l'*accroissement différentiel* de la proportion d'actifs en arrêt entre 2019 et 2020. On remarque que cet accroissement est plus le plus fort pour les catégories d'employés et les professions intermédiaires, pour lesquelles il représente environ 1 point de pourcentage. Il est un peu plus faible pour les ouvriers, et beaucoup plus faible pour les autres catégories d'actifs.

In [ ]:
# Calcul de la proportion d'actifs en arrêt de travail selon les années en fonction de NAF agrégée
df_naf_annee = exposition.exposition_annee(eec_actifs, var="NAFR", year_col="ANNEE")

# Représentation graphique de l'évolution de la proportion d'arrêts maladie par secteur d'activité
exposition.plot_evolution_proportion(
    df=df_naf_annee,
    year_col="ANNEE",
    group_col="NAFR",
    value_col="proportion_rabs2",
    title="Évolution de la proportion d'absences au travail pour congé maladie selon le secteur d'activité",
    ylabel="Pourcentage d'actifs absents au travail pour congé maladie",
    )

Ce graphique est peu lisible, car la variable de secteur d'activité NAF comporte trop de catégories pour représenter leurs évolutions aisément, même à un niveau agrégé. On peut toutefois remarquer que la proportion d'actifs en arrêt pour maladie augmente davantage pour certaines catégories, par exemple pour les catégories `NAF == E` ce qui correspond au secteur de la distribution d'eau et au traitement des déchets, ou `NAF == Q` qui regroupe les professions du secteur de la santé et de l'action sociale.

Enfin, l'évolution des arrêts maladie est également variable selon le sexe, comme l'indique le graphique suivant :

In [ ]:
# Calcul de la proportion d'hommes et de femmes en arrêt de travail selon les années 
df_sexe_annee = exposition.exposition_annee(eec_actifs, var="SEXE_label", year_col="ANNEE")

# L'évolution du nombre d'arrêts de travail pour congé maladie en fonction du sexe
exposition.plot_evolution_proportion(
    df=df_sexe_annee,
    year_col="ANNEE",
    group_col="SEXE_label",
    value_col="proportion_rabs2",
    title="Évolution de la proportion d'absences au travail pour congé maladie selon le sexe",
    ylabel="Pourcentage d'actifs absents au travail pour congé maladie",
    colors= {"Homme": "#7A57FF","Femme": "#C869FF"}
)

Les femmes sont en moyenne davantage en arrêt que les hommes, et la proportion d'arrêt a également augmenté davantage en 2020 pour les femmes que pour les hommes. On pourrait donc envisager d'utiliser la variable de sexe comme une variable explicative, notamment car les recherches médicales semblent montrer que les femmes développent davantage des covid-longs que les hommes. Cela aurait toutefois l'inconvénient de masquer les effets structurels de l'exposition différenciée au Covid-19 selon le sexe en 2020, comme le suggèrent les deux graphiques précédents : les femmes sont en effet davantage représentées parmi les secteurs les plus exposées, ce que l'on montre dans la section suivante.

### Calcul d'un score d'exposition par catégorie

Une deuxième façon de vérifier la pertinence de notre proxy est de comparer l'évolution du nombre d'arrêts de travail pour congé maladie en 2020 aux autres années. En lien avec notre problématique, on peut directement représenter cette évolution en fonction de nos variables d'intérêt, par exemple le sexe et la PCS. 

### Contrôle par le nombre d'arrêt de travail précédant le Covid

Attention cependant, il serait erroné de considérer la seule proportion d'arrêts maladie en 2020 comme proxy de la contamination au Covid-19. En effet, comme le montre la Figure [insérer nom Figure PCS / arrêts maladie toute la période], il y a des différences structurelles entre secteurs d'activité dans la proportion d'actifs en arrêt maladie. Du fait de leurs conditions de travail [insérer référence ?] (pénibilité, risques d'accident...), les ouvriers et les employés sont par exemple systématiquement plus en arrêt maladie que les autres. C'est donc également le cas en 2020, sans que cela reflète une plus forte exposition au Covid. Nous nous intéresserons donc plutôt à la différence, pour chaque secteur, entre la proportion d'actifs en arrêt maladie en 2020 et celle en 2019, une différence positive reflétant donc une augmentation des arrêts maladies, que nous attribuons en partie au Covid.

In [ ]:
# Fonction pour calculer la différence de proportion d'actifs en arrêts de travail durant la semaine de référence entre deux années selon les catégories d'une variable donnée (ici 'CSE')
exposition.exposition_diff(eec_actifs, year1 = 2019, year2 = 2020, var = 'CSE')
# exposition.exposition_diff(eec_actifs, year1 = 2019, year2 = 2020, var = 'NAFG038UN')

### Cohérence avec la littérature

[insérer catégories exposées dans d'autres enquêtes, la similarité avec celles qu'on a et des pistes d'explication]

### Calcul du score d'exposition 

[Justifier la combinaison des PCS détaillées et des secteurs d'activité NAF]
[Expliquer qu'ils se recoupent en partie mais pas totalement ; on enlèvera plus tard les catégories regroupant mécaniquement trop peu de gens]
[Expliquer et justifier notre score d'exposition comme sommme des deux scores d'exposition]

[Repartir de notre cadre = absence de données individuelles : nos individus c'est les secteurs]

In [ ]:
# Ajout de la variable de score d'exposition à la table eec_actifs
eec_actifs = exposition.score_exposition(eec_actifs, var_list = ['CSE', 'NAFG038UN'])
eec20 = eec_actifs[eec_actifs['ANNEE'] == 2020]

In [ ]:
eec_actifs.head()

In [ ]:
exposition.plot_score_exposition(eec20)
# pondérer ce code ?

In [ ]:

importlib.reload(exposition)

## Un risque d'exposition socialement différencié ?

### Le lien entre l'exposition au Covid du secteur d'activité et le sexe, la PCS et le caractère public de l'emploi

#### Statistiques descriptives

In [ ]:
exposition.plot_freq_exposition(eec20, by='SEXE_label',figsize=(8,6))

In [ ]:
exposition.plot_freq_exposition(eec20, by='CSER_label',  figsize=(11,7))
# là j'ai repris un code couleur utilisé par l'Insee pour les PCS mais c'est pas dans le style de notre chartre couleur pastel

[Discuter de l'importance des différences présentes dans les résultats, en lien avec la manière dont a été calculée le score d'exposition : car puisqu'il a été construit uniquement sur la base des secteurs les plus exposés, il y a déja un regroupement via le secteur d'activité, qui explique peut être pk y a aucun cadre dans les secteurs les plus exposés (choisis par tertiles), même s'il doit y avoir un certain % de cadres parmi les plus exposés] => en gros la manière précédente de constuire le score accentue peut être les différences selon une variable liée au secteur d'activité (ici la PCS) 

In [ ]:
exposition.plot_freq_exposition(eec20, by='PUB_label')
# Exclues du graphique personnes non actives occupées et actifs occupés indépendants STAT2 distinct de 2

#### Modèles de régression linéaire

Comment ces trois variables explicatives (le sexe, la PCS, et le caractère public ou non de l'emploi) interagissent-elles pour prédire le score d'exposition au Covid, associé aux combinaisons de secteurs d'activité ? 

In [ ]:
importlib.reload(exposition)

In [ ]:
# On vérifie que les variables qu'on souhaite utiliser comme régresseurs soient bien catégorielles
categorical_vars = ["SEXE_label", "CSER_label", "PUB_label"]
reference_dict = {"SEXE_label": "Homme", "CSER_label": "Cadres et professions intellectuelles supérieures", "PUB_label": "Secteur privé"}

df_coef, modele_robust = exposition.regression_exposition(eec20, categorical_vars=categorical_vars, reference_dict=reference_dict)
# print(modele_robust.summary())


In [ ]:
exposition.plot_regression_exposition(df_coef)

QUESTIONNER LA PERTINENCE D'UN MODELE DE REGRESSION LINEAIRE ET PAS LOGIT ORDONNE
inconvenient: certaines hypothèses du modèle linéaire: données continues non bornées => on peut prédire un score > 4 (à l'inverse des hommes cadres du privé, les femmes employées travaillant dans les hôpitaux publics ont un "score moyen" de 4.1978, toutes choses égales par ailleurs)
avantage (ou limites de l'inconvénient) : pas absurde car on compare des catégories seulement + de manière plus exploratoire que prédictive 

**Interprétation**

Intercept à 1.2 : un homme cadre du privé a un score moyen d’exposition de 1.2 sur 0–4
Par rapport à ce profil de référence, toutes choses égales par ailleurs, toutes les autres modalités augmentent significativement le score d'exposition au Covid. 

Les femmes ont un score moyen 0.197 point plus élevé que les hommes (référence), toutes choses égales par ailleurs (effet notable mais modéré). L'importance réduite de cet effet par rapport aux statistiques descriptives suggère que le lien fort existant entre sexe et exposition au Covid passe en partie par des différences de secteurs d'activité. 

Les professions moins qualifiées sont beaucoup plus exposées que les cadres supérieurs. Les employés ont un score moyen 1.33 point plus élevé que les cadres supérieurs. Les ouvriers et les professions intermédiaires ont un score moyen environ 0.85 point plus élevé que les cadres supérieurs. 

Enfin, le secteur public est beaucoup plus exposé, particulièrement les hôpitaux, qui ont une exposition moyenne 1.44 point plus élevée.

Le R^2 du modèle est de 0.40, ce qui signifie que 40% de la variance du score est expliquée par nos variable, ce qui est extrêmement fort en sciences sociales. Ces trois variables sont donc assez prédictives, même si rien ne garantit que les liens observés soient de nature causale.

[TRANSITION AVEC METHODE DES DOUBLES DIFFERENCES POUR TENTER DE METTRE A JOUR DES EFFETS CAUSAUX ?]

<a id="modelisation"></a>

# Covid long et réduction de l'offre de travail

Nous analysons à présent les effets des absences pour raisons de santé sur la participation à la population active à l’aide d’une étude d’événement.

## Doubles-différences

Afin d'évaluer l'impact de la pandémie sur les différents secteurs économiques, nous mobilisons une stratégie de différences-en-différences (DiD) en comparant les secteurs exposés au Covid-19 aux secteurs non exposés. Nous estimons dans un premier temps la spécification suivante :
$$
Y_{it}=\alpha + \tau_\text{DiD}(\mathbf{1}\{t\geq 2020\}\times\mathbf{1}\{\text{exposé}_i=1\}) + \mathbf{X}_{it} \Lambda_k + \gamma_i + \delta_t + \varepsilon_{it}
$$
où $Y_{it}$ représente le niveau d’activité (ou de participation) du secteur $i$ à la période $t$. La variable $\text{Exposé}_i$ est une indicatrice valant 1 si le secteur appartient au groupe traité, et $\text{Post}_t$ est une indicatrice valant 1 pour les années postérieures à 2020. Le vecteur $\mathbf{X}_{it}$ contient un ensemble de caractéristiques sectorielles variant dans le temps. Les termes $\gamma_i$ et $\delta_t$ désignent respectivement des effets fixes de secteur et de période, capturant les chocs temporels globaux et l'hétérogénéité sectorielle qui ne varie pas avec le temps. Dans ce cadre, le coefficient d'intérêt $\tau_{\text{DiD}}$ mesure l'effet moyen du traitement sur les traités (ATT), c'est-à-dire l'impact différentiel moyen de l'exposition au Covid-19 sur la variable de résultat des secteurs traités après 2020. Afin de garantir des inférences statistiques valides en présence d'une potentielle autocorrélation des erreurs au sein de chaque secteur d'activité, nous utilisons des erreurs standards robustes au niveau de chaque secteur.

Notre hypothèse d’identification suppose que, hormis l'épisode d’absence pour raisons de santé en 2020, il n’existerait aucune différence moyenne de taux de participation entre les secteurs, une fois que l'on a contrôlé pour leurs caractéristiques initiales $\mathbf{X}_{it}$ ainsi que pour le secteur et la période (effets fixes $\gamma_i$ et $\delta_t$). Dans ce cadre, le terme $\tau_\text{DiD}$ capture l'effet propre de l'exposition au Covid-19 sur la participation au marché du travail des salariés du secteur $i$ après 2020 en moyenne.

### Préparation des données

Dans notre stratégie d'identification, nous utilisons une approche au niveau de chaque groupe $i$ construit par l'intersection d'un secteur économique NAF et d'une catégorie socio-professionnelle.

In [ ]:
# Données
df = eec_actifs

# Agrégation par CSP x NAF de la proportion d'individus ayant déclaré un arrêt maladie
## Création de la variable "groupe" en combinant CSE et NAFG038UN
df['groupe'] = df.groupby(['CSE', 'NAFG038UN']).ngroup()

## Création d'une indicatrice d'arrêt maladie
df['is_RABS_2'] = (df['RABS'] == '2')

Néanmoins, certaines de ces combinaisons n'ont pas beaucoup de sens.

In [ ]:
## Montrer le tableau des effectifs inférieurs à 20 que l'on vire pour montrer que l'on sait manipuler et visualiser les données
check = df.groupby(['groupe']).filter(
    lambda x: (x.groupby('ANNEE').size() < 20).all()
    )

check.head()

On ne conserve donc que celles qui comptent plus de vingt actifs pour toutes les années de la période d'analyse, puis on agrége les données.

In [ ]:
## Sélection des CSP x NAF avec au moins vingt actifs pour toutes les années de la période
df = df.groupby(['groupe']).filter(
    lambda x: (x.groupby('ANNEE').size() >= 20).all()
    )

## Agrégation par CSP x NAF
df = df.groupby(['groupe', 'ANNEE']).agg(
    sh_arret=('is_RABS_2', 'mean'),
    n_individus=('is_RABS_2', 'size'),
    cat_expose=('score_exposition', 'mean'),
).reset_index()

## Évaluation des groupes obtenus
n_groupes = len(df[['groupe']].drop_duplicates())
print(f"Nombre de combinaisons CSP x secteur : {n_groupes}")

## Affichage du résultat
df.head()

Pour pouvoir identifier un effet, il est nécessaire de disposer de variabilité dans les données selon les groupes constitués et les années.

In [ ]:
# On regarde la distribution des arrêts maladie
plt.figure(figsize=(10, 6))
sns.histplot(df['sh_arret'], bins=30, color='#7A57FF')
plt.title('Distribution de la proportion d\'arrêts maladie (sh_arret)')
plt.xlabel('Proportion d\'arrêts')
plt.ylabel('Nombre de groupes x année (Fréquence)')
plt.show()

On peut finalement préparer les données pour l'analyse

In [ ]:
# Préparation des données pour l'analyse
## Construction des variables d'exposition 1{exposé == 1}
print(df['cat_expose'].unique())
df['expose_1'] = df['cat_expose'].isin([4]).astype(int)
df['expose_2'] = df['cat_expose'].isin([3, 4]).astype(int)
df['expose_3'] = df['cat_expose'].isin([2, 3, 4]).astype(int)

## Vérification
df.head()

In [ ]:
# Données propres aux doubles-différences
## Création de la variable Post = 1{t >= 2020}
df['post'] = (df['ANNEE'] >= 2020).astype(int)

## Création du terme d'interaction 1{t >= 2020} x 1{exposé == 1} pour chaque définition du traitement
df['traitement_1'] = df['post'] * df['expose_1']
df['traitement_2'] = df['post'] * df['expose_2']
df['traitement_3'] = df['post'] * df['expose_3']

## Vérification
df.head(15)

In [ ]:
# Contrôle des données
df_reg = df.set_index(['groupe', 'ANNEE'])
print(f"Dimension des données : {df_reg.shape}")
print(f"Nombre de groupes uniques : {df_reg.index.get_level_values('groupe').nunique()}")
print(f"Nombre d'années uniques : {df_reg.index.get_level_values('ANNEE').nunique()}")

variance_y = df_reg['sh_arret'].var()
print(f"Variance de la variable sh_arret : {variance_y}")

n_nans = df_reg[['sh_arret', 'traitement_1']].isna().sum().sum()
print(f"Nombre de valeurs manquantes : {n_nans}")

### Résultats

Une fois les données structurées convenablement, on estime notre spécification en double-différences.

In [ ]:
# Estimation (DiD canonique)

## Définition de l'index pour les données de panel dans linearmodels
df_reg = df.set_index(['groupe', 'ANNEE'])

## Définition de la variable expliquée Y et de la variable dépendante X
Y = df_reg['sh_arret']
X = df_reg[['traitement_1']] 

# Configuration du modèle
modele = PanelOLS(
    Y,                      # Part d'arrêts maladie
    X,                      # Traitement
    entity_effects=True,    # Capte les différences de niveau entre les groupes (remplace 'expose')
    time_effects=True       # Capte l'évolution commune à tous les groupes (remplace 'post')
    )

# Estimation avec erreurs-types clusterisées (Robustes à l'autocorrélation intra-groupe)
resultat = modele.fit(cov_type='clustered', cluster_entity=True)

## Résultat
print(resultat.summary)

On vérifie la robustesse de nos résultats à des définitions alternatives de l'exposition au Covid-19.

In [ ]:
# Estimation pour des traitements alternatifs
liste_traitements = ['traitement_1', 'traitement_2', 'traitement_3']

for t in liste_traitements:
    print(f"\n{'='*40}")
    print(f"Estimation pour : {t}")
    print(f"{'='*40}")
    
    # On sélectionne la colonne de traitement spécifique
    X = df_reg[[t]]
    
    # On lance le modèle
    modele = PanelOLS(df_reg['sh_arret'], X, entity_effects=True, time_effects=True)
    resultat = modele.fit(cov_type='clustered', cluster_entity=True)
    
    # On affiche le coefficient estimé, la p-valeur et le R-carré
    print(resultat.params)
    print(f"P-value: {resultat.pvalues[0]:.3f}")
    print(f"R-squared (Within): {resultat.rsquared_within:.2f}")

## Étude d'évènement

Pour explorer la dynamique temporelle des effets et tester la validité de notre hypothèse d'identification, nous généralisons l'approche précédente par une étude d'événement (event study) :
$$
Y_{it} = \alpha + \sum_{\substack{j \in T \\ j \neq -1}} \tau_j (\mathbf{1}\{t=j\} \times \mathbf{1}\{\text{exposé}_i=1\}) + \gamma_i + \delta_t + \varepsilon_{it}
$$
où $\mathbf{1}\{t=j\}$ est une indicatrice égale à 1 pour l'année relative $j\in T=\{-10,\cdots,+4\}$ de sorte que les coefficients $\tau_j$, pour $j \geq 0$, capturent les effets dynamiques de l'exposition au Covid-19 $j$ période(s) après 2020. Nous fixons l'année 2019 comme période de référence, de sorte que $\tau_{-1}=0$.

Cette spécification dynamique permet de vérifier l'hypothèse fondamentale de tendances parallèles. Celle-ci postule qu'en l'absence de pandémie, l'évolution de la variable $Y_{it}$ aurait été identique pour les secteurs exposés et non exposés. La nullité des coefficients $\tau_j$ pour $j \leq -2$ constitue un test de cette hypothèse.

Enfin, il convient de noter que notre cadre d'analyse repose sur une expérience naturelle où le traitement est administré de manière synchrone (en 2020). L'absence de décalage temporel dans l'adoption du traitement (staggered adoption) nous permet de nous affranchir des biais soulignés par Goodman-Bacon ([2021](https://www.google.com/search?client=safari&rls=en&q=Goodman-Bacon+A+(2021)+Difference-in-differences+with+variation+in+treatment+timing.+J+Econom+225(2)%3A254%E2%80%93277&ie=UTF-8&oe=UTF-8)). Par conséquent, l'estimateur par effets fixes bidirectionnels (TWFE) demeure convergent et ne nécessite pas le recours aux estimateurs robustes développés récemment pour les traitements échelonnés (e.g., Callaway et Sant’Anna, [2021](https://www.google.com/search?client=safari&rls=en&q=Callaway+B%2C+Sant%E2%80%99Anna+PH+(2021)+Difference-in-differences+with+multiple+time+periods.+J+Econom+225(2)%3A200%E2%80%93230&ie=UTF-8&oe=UTF-8) ; de Chaisemartin et d'Haultfœuille, [2022](https://www.nber.org/papers/w29873); Borusyak et al., [2024](https://academic.oup.com/restud/article/91/6/3253/7601390)).

### Préparation des données

On commence par préparer les données en construisant les indicatrices $\mathbf{1}\{t=j\}$ et $\mathbf{1}\{\text{exposé}_i=1\}$

In [ ]:
# Données propres aux études d'évènement
## Création de la variable relative j "annee_relative" centrée sur 2020
df['annee_relative'] = (df['ANNEE'] - 2020).astype(int)

## Création des indicatrices 1{t == j}, sauf pour l'année relative de référence j == -1
df = (
    pd.get_dummies(df, columns=['annee_relative'], prefix='ttt')
    # On remplace le tiret '-' par 'm' dans les noms de colonnes
    .rename(columns=lambda x: x.replace('-', 'm'))
    # On supprime la colonne de référence (ttt_m1 correspond à j = -1)
    .drop(columns='ttt_m1', errors='ignore')
    # On définit l'index sur les identifiants de groupe et de temps
    .set_index(['groupe', 'ANNEE'])
)

## Création des termes d'interaction 1{t == j} x 1{exposé == 1}
var_traitement = 'expose_1'
dummies_annees_relatives = [c for c in df.columns if c.startswith('ttt')]
vars_interaction_event_study = []
for col in dummies_annees_relatives:
    # On multiplie l'indicatrice de temps 1{t == j} par le statut de traitement 1{exposé == 1}
    nom_colonne = f'{var_traitement}_{col}'
    df[nom_colonne] = df[var_traitement] * df[col]
    vars_interaction_event_study.append(nom_colonne)

### Résultats

On estime ensuite notre spécification dynamique.

In [ ]:
# Estimation (Étude d'évènement)

## Définition de la variable expliquée Y et des variables d'interaction X
Y = df['sh_arret']
X = df[vars_interaction_event_study] # On met toutes les interactions (leads et lags)

## Configuration du modèle
modele = PanelOLS(
    Y,                      # Variable expliquée: part d'arrêts maladie Y_it
    X,                      # Variable explicatives: 1{t == j} x 1{exposé == 1}
    entity_effects=True,    # Effets-fixes groupes: gamma_i (capte les différences de niveau entre les groupes)
    time_effects=True       # Effets-fixes années: delta_t (capte l'évolution commune à tous les groupes)
)

## Estimation avec erreurs-types clusterisées (Robustes à l'autocorrélation intra-groupe)
resultat = modele.fit(cov_type='clustered', cluster_entity=True)

## Résultat
print(resultat.summary)

Pour rendre la lecture des résultats plus lisible, on les représente sous forme graphique. Chacun des points représente un $\tau_j$, et est associé à son intervalle de confiance à 95 %.

In [ ]:
# Représentation graphique des résultats

## Récupération des coefficients estimés et des intervalles de confiance
est = resultat.params
ic = resultat.conf_int()

## Enregistrement dans un dataframe
df_plot = pd.DataFrame({
    'coef': est,
    'lower': ic.iloc[:, 0],
    'upper': ic.iloc[:, 1]
})

## Extraction des variables d'années relatives à partir du nom des variables
def extract_time(col_name):
    suffix = col_name.split('_')[-1] # Prend "m2" ou "0"
    if 'm' in suffix:
        return -int(suffix.replace('m', ''))
    return int(suffix)

df_plot['temps_relatif'] = [extract_time(idx) for idx in df_plot.index]

## Ajout manuel de l'année de référence (-1) où le coef est 0 par définition
row_ref = pd.DataFrame({'coef': [0], 'lower': [0], 'upper': [0], 'temps_relatif': [-1]})
df_plot = pd.concat([df_plot, row_ref]).sort_values('temps_relatif')

## Graphique
plt.figure(figsize=(10, 6))

plt.axvspan(-0.5, 0.5, color='grey', alpha=0.2, label='Année du traitement', zorder=0)

plt.errorbar(
    df_plot['temps_relatif'], 
    df_plot['coef'], 
    yerr=[df_plot['coef'] - df_plot['lower'], df_plot['upper'] - df_plot['coef']],
    fmt='o',
    color='#7A57FF', 
    ecolor='#CABCFF',
    elinewidth=4,
    capsize=0,
    label='Coefficients estimés et intervalle de confiance à 95%'
)

plt.axhline(0, linestyle='dashed', color='grey')

plt.title('Étude d\'évènement : Impact sur les arrêts maladie')
plt.xlabel('Années relatives au traitement (0 = 2020)')
plt.ylabel('Effet estimé sur l\'activité')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

<a id="conclusion"></a>

# Conclusion